# 相似度计算字符串相似度计算
功能描述：计算一个字符串与一组字符串的相似程度，返回Top N相似度的字符串及其相似度

相似度计算方法：
- 1. tokens之间的余弦相似度
- 2. tokens之间经由预训练Bert embembedding之后的余选相似度
- 3. tokens的TF-IDF余弦相似度 (文档相似度)

In [1]:
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

PRETRAINED_BERT = SentenceTransformer('bert-base-nli-mean-tokens')

def get_potential_string(raw_string, reference, Top_N, stemming=True,mode = 'direct'):
    '''
    :param raw_string: 需要计算相似程度的string
    :param reference: 被计算相似程度的参考listing, list of string
    :param Top_N: 返回Top N个相似结果
    :param stemming:  是否进行词干抽取，默认进行词干抽取
    :param mode: 可选择的相似度算法,目前有direct, Bert, TF-IDF 三个功能
    :return: (原string,pd.df)的集合
    '''
    # 如果启用了词干抽取，则创建一个SnowballStemmer实例，用于进行词干抽取
    assert mode in ['direct','Bert','TF-IDF'],'请使用direct, Bert, TF-IDF'
    if stemming:
        stemmer = SnowballStemmer(language='english')
        # 将输入字符串和参考字符串列表进行词干抽取
        raw_string_stemming = ' '.join([stemmer.stem(word) for word in raw_string.split()])
        reference_stemming = [' '.join([stemmer.stem(word) for word in category.split()]) for category in reference]
    else:
        raw_string_stemming = raw_string
        reference_stemming = reference

    if mode == 'direct':
        # 创建一个CountVectorizer实例，用于将文本转换为矢量表示
        vectorizer = CountVectorizer()
        # 将输入字符串和参考字符串列表传递给vectorizer，生成文本的矢量表示
        # X[0]是输入字符串的矢量表示，X[1:]是参考字符串列表的矢量表示
        X = vectorizer.fit_transform([raw_string_stemming] + reference_stemming)
        # 计算输入字符串和参考字符串列表中每个元素之间的余弦相似度得分
        sim_scores = cosine_similarity(X[0], X[1:])

    elif mode == 'Bert':
        X = PRETRAINED_BERT.encode([raw_string_stemming] + reference_stemming)
        # 计算输入字符串和参考字符串列表中每个元素之间的余弦相似度得分,这里的首个字符是list of list
        sim_scores = cosine_similarity([X[0]], X[1:])

    elif mode == 'TF-IDF':
        tfidf_vec = TfidfVectorizer()
        X  = tfidf_vec.fit_transform([raw_string_stemming] + reference_stemming)
        sim_scores = cosine_similarity(X[0], X[1:])

        
    # 将字符串与每个参考字符串及其相应的相似度得分存储在一个pandas DataFrame中作为输出结果
    Similarity_Result = (
        pd.DataFrame(sim_scores).T
        .assign(Potential_String=reference)
        .sort_values(by=0, ascending=False)
        .nlargest(Top_N, columns=0)
        .rename(columns={0: 'Similarity_Score'})
    )
    return raw_string,Similarity_Result

c:\Users\Zhang\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 读取商品的类目树数据
类目树 即我们需要比对的参考字符串列表

In [2]:
from QJBSL.dataset import load_Category_relationships
category_tree = load_Category_relationships(format='Long')
category_tree

,id,产品品类名称,品类别名,Value
0,1,Bumper,品类别名_1,Bumper
1,2,Headlight,品类别名_1,Head Light
2,3,Door Mirror,品类别名_1,Towing Mirror
3,4,Control Arm,品类别名_1,Suspension Stabilizer Bar
4,5,Grille,品类别名_1,Grill
...,...,...,...,...
57568,529,Alternator Other Part,品类别名_24,Alternator Bushing
60048,529,Alternator Other Part,品类别名_25,Alternator Tester
62528,529,Alternator Other Part,品类别名_26,Alternator Cable
65008,529,Alternator Other Part,品类别名_27,Alternator Cover


# 直接计算余弦相似度的效果
token化之后直接计算余弦相似度，简单暴力，在品类匹配任务中速度快，效果好。完美带使用in进行字符串比对。

In [3]:
# 随便从电商网站上找了一个listing的标题作为我们的输入字符串
txt = 'NEW 200 AMP Alternator fits Nissan Pathfinder Infiniti G35 QX4 2003-2004'

# 调用get_potential_string函数，计算输入字符串与参考字符串列表中每个元素之间的相似度得分
result = get_potential_string(txt,                                      # 输入字符串  
                              category_tree.get('Value').to_list(),     # 参考字符串列表
                              Top_N = 6,                                # 返回前6个相似结果
                              mode='direct',                            # 使用direct模式
                              stemming=True)                            # 对字符串进行词干抽取
result

('NEW 200 AMP Alternator fits Nissan Pathfinder Infiniti G35 QX4 2003-2004',
       Similarity_Score     Potential_String
 42            0.288675           Alternator
 2927          0.288675    Alternator O-Ring
 2939          0.204124   Alternator Coupler
 2906          0.204124  Alternator Resistor
 2914          0.204124      Alternator Bolt
 2858          0.204124    Alternator Pulley)

# 计算TF-IDF相似度

>   TF-IDF是一种用于信息检索与文本挖掘的常用加权技术。tf-idf是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。tf-idf加权的各种形式常被搜索引擎应用，作为文件与用户查询之间相关程度的度量或评级。除了tf-idf以外，互联网上的搜索引擎还会使用基于链接分析的评级方法，以确定文件在搜索结果中出现的顺序。

<br><p style="text-align:right;"> -- <cite>[TF-IDF - 维基百科](https://zh.wikipedia.org/zh-cn/Tf-idf)</cite></p>

在品类匹配的任务中,tf-idf可以较好的识别到关键词。但考虑到tf-idf对于重要的词汇的权重是根据文档数据集动态而变化的，所以其更加适合用于一批商品的标题匹配任务。



In [4]:
# 随便从电商网站上找了一个listing的标题作为我们的输入字符串
txt = 'NEW 200 AMP Alternator fits Nissan Pathfinder Infiniti G35 QX4 2003-2004'

# 调用get_potential_string函数，计算输入字符串与参考字符串列表中每个元素之间的相似度得分
result = get_potential_string(txt,                                      # 输入字符串  
                              category_tree.get('Value').to_list(),     # 参考字符串列表
                              Top_N = 6,                                # 返回前6个相似结果
                              mode='TF-IDF',                            # 使用TF-IDF模式
                              stemming=True)                            # 对字符串进行词干抽取
result

('NEW 200 AMP Alternator fits Nissan Pathfinder Infiniti G35 QX4 2003-2004',
       Similarity_Score   Potential_String
 42            0.195594         Alternator
 2927          0.195594  Alternator O-Ring
 1684          0.188857   Radiator Fitting
 2244          0.176377       Tube Fitting
 1164          0.176377     Grease Fitting
 1599          0.175144       Pipe Fitting)

# 计算Bert Embedding之后的余弦相似度
调用Bert的计算成本要比直接计算余弦相似度高很多，但在短字符串相似度匹配中的效果反而差了很多。可以期待其在长字符串匹配中的效果。

In [5]:
# 随便从电商网站上找了一个listing的标题作为我们的输入字符串
txt = 'NEW 200 AMP Alternator fits Nissan Pathfinder Infiniti G35 QX4 2003-2004'

# 调用get_potential_string函数，计算输入字符串与参考字符串列表中每个元素之间的相似度得分
result = get_potential_string(txt,                                      # 输入字符串  
                              category_tree.get('Value').to_list(),     # 参考字符串列表
                              Top_N = 6,                                # 返回前6个相似结果
                              mode='Bert',                              # 使用Bert Embedding
                              stemming=True)                            # 对字符串进行词干抽取
result

('NEW 200 AMP Alternator fits Nissan Pathfinder Infiniti G35 QX4 2003-2004',
       Similarity_Score              Potential_String
 2622          0.576172              Fuel Door Bumper
 2874          0.564890                Fender Grommet
 1046          0.532475                   Fender Seal
 1044          0.529691              Fender Outrigger
 2857          0.525722                 Fender Gasket
 70            0.523874  Fuel Injection Throttle Body)